### Create the below table in Azure SQL DB before proceeding with the below steps

<p>   CREATE TABLE CUSTOMER 
        (
<br>    C_CUSTKEY		INT,
<br>	C_NAME			VARCHAR(25),
<br>	C_ADDRESS		VARCHAR(40),
<br>	C_NATIONKEY		SMALLINT,
<br>	C_PHONE			CHAR(15),
<br>	C_ACCTBAL		DECIMAL,
<br>	C_MKTSEGMENT	CHAR(10),
<br>	C_COMMENT		VARCHAR(117)
        );</p>

In [0]:
from pyspark.sql.types import *

In [0]:
# Details about connection string
logicalServername = "devlogicalserver.database.windows.net"
databaseName = "demoDB"
tableName = "CUSTOMER"
userName = "sqladmin"
password = "Password@12345" # Please specify password here
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(logicalServername, 1433, databaseName)
connectionProperties = {
  "user" : userName,
  "password" : password,
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


In [0]:
#Creating a schema which can be passed while creating the dataframe
cust_schema = StructType([
    StructField("C_CUSTKEY", IntegerType()),
    StructField("C_NAME", StringType()),
    StructField("C_ADDRESS", StringType()),
    StructField("C_NATIONKEY", ShortType()),
    StructField("C_PHONE", StringType()),
    StructField("C_ACCTBAL", DecimalType(18,2)),
    StructField("C_MKTSEGMENT", StringType()),
    StructField("C_COMMENT", StringType())
])

In [0]:
# Reading customer csv files in a dataframe. This Dataframe will be written to Customer table in Azure SQL DB
df_cust= spark.read.format("csv").option("header",True).schema(cust_schema).load("dbfs:/mnt/Gen2/Customer/csvFiles")

### Reading and writing data from and to Customers table using JDBC SQLServer Drivers

In [0]:
#Reading the table we have created and we shouldn;t see any rows in the dataframe
df_jdbcRead= spark.read.jdbc(jdbcUrl,  
                   table=tableName, 
                   properties=connectionProperties)

display(df_jdbcRead.printSchema())

root
-- C_CUSTKEY: integer (nullable = true)
-- C_NAME: string (nullable = true)
-- C_ADDRESS: string (nullable = true)
-- C_NATIONKEY: short (nullable = true)
-- C_PHONE: string (nullable = true)
-- C_ACCTBAL: decimal(18,2) (nullable = true)
-- C_MKTSEGMENT: string (nullable = true)
-- C_COMMENT: string (nullable = true)

In [0]:
# Writing the dataframe which has all the csb read from ADLS Gen-2 folder to Customer table in Azure SQL Db
df_cust.write.jdbc(jdbcUrl,  
                   mode ="append", 
                   table=tableName, 
                   properties=connectionProperties)

In [0]:
#Lets read the table after loading the data to count the number of rows
df_jdbcRead= spark.read.jdbc(jdbcUrl,  
                   table=tableName, 
                   properties=connectionProperties)

#Counting number of rows
df_jdbcRead.count()

Out[51]: 150000

### Reading and writing data from and to Customers table using Apache Spark connector for Azure SQL

In [0]:
server_name = f"jdbc:sqlserver://{logicalServername}"
database_name = "demoDB"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "Customer"
username = "sqladmin"
password = "Password@12345" # Please specify password here

### Reading data from Customers table using Apache Spark connector

In [0]:
sparkconnectorDF = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()

In [0]:
display(sparkconnectorDF.printSchema())

root
-- C_CUSTKEY: integer (nullable = true)
-- C_NAME: string (nullable = true)
-- C_ADDRESS: string (nullable = true)
-- C_NATIONKEY: short (nullable = true)
-- C_PHONE: string (nullable = true)
-- C_ACCTBAL: decimal(18,2) (nullable = true)
-- C_MKTSEGMENT: string (nullable = true)
-- C_COMMENT: string (nullable = true)

In [0]:
display(sparkconnectorDF.limit(10))

C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
135496,Customer#000135496,k0 TUmPbSPhzs08cfI9ImJH6gaMXafD,0,10-506-222-3529,8698.5,AUTOMOBILE,thely express instructions. slyly express sentiments cajole deposits-- fur
140941,Customer#000140941,Dgjzz1ZMmbCcjsbXEiQP,0,10-913-403-1164,8186.2,AUTOMOBILE,pecial excuses. fluffily final excuses nag carefully above the pin
143890,Customer#000143890,WvuEN zg8F5mFOuFO8QgxR,0,10-835-720-4491,9143.12,MACHINERY,"st the final, express excuses. furiously regular platelets haggle furiously pending, pending courts. r"
145966,Customer#000145966,",3JKjCVmEoK3R0lagbPg,dXRQB6X,J,Z",0,10-115-248-1195,78.5,MACHINERY,ly regular pinto beans sleep along the blithely unusual packag
144925,Customer#000144925,knp9xB2RzFTpw4GUiDpHebR Z,0,10-406-869-7532,-955.51,FURNITURE,"ndencies. final, even accounts haggle quickly furiously unusual realms. express, special platelets"
142286,Customer#000142286,vVmsQfdo5FcMVnFEm,0,10-376-255-1883,3562.27,FURNITURE,ongside of the blithely quick foxes. evenly even ideas are car
130367,Customer#000130367,d5byybp0cXAcgWPQMmqPyY,0,10-837-829-2496,4264.46,FURNITURE,nts wake blithely busy deposits. quickly bold packages sublate carefully slyly bol
142641,Customer#000142641,v r8TtQ70JtFz9y8AuPWkfCaiSG,0,10-782-240-2178,3775.19,FURNITURE,furiously. carefully regular deposits affix blithely at the slyly regular deposits! slyly silent theodolites
141468,Customer#000141468,3JwrybprIBx,0,10-592-815-7985,3969.22,FURNITURE,uriously silent ideas are fluffily. furiously pending
135148,Customer#000135148,rLZ6O1DtaJJRPEYWxbSNIJRPTqpwg05TIGJe,0,10-403-887-3113,2502.51,HOUSEHOLD,s. quickly ironic requests poach carefully. unusual accounts print quickly. slyl


In [0]:
#Truncating the table and loading the data. If we need to preserve the indexes created in this table then when should use the option("truncate",True) which will avoid dropping the index
try:
  df_cust.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("overwrite") \
    .option("truncate",True) \
    .option("url", url) \
    .option("dbtable", tableName) \
    .option("user", userName) \
    .option("password", password) \
    .save()
except ValueError as error :
    print("Connector write failed", error)

In [0]:
#Appending records to the existing table
try:
  df_cust.write \
    .format("com.microsoft.sqlserver.jdbc.spark") \
    .mode("append") \
    .option("url", url) \
    .option("dbtable", tableName) \
    .option("user", userName) \
    .option("password", password) \
    .save()
except ValueError as error :
    print("Connector write failed", error)

In [0]:
#Lets read the data from the table
sparkconnectorDF = spark.read \
        .format("com.microsoft.sqlserver.jdbc.spark") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("user", username) \
        .option("password", password).load()

In [0]:
display(sparkconnectorDF.limit(10))

C_CUSTKEY,C_NAME,C_ADDRESS,C_NATIONKEY,C_PHONE,C_ACCTBAL,C_MKTSEGMENT,C_COMMENT
124113,Customer#000124113,"l,nnn8dPsWzByhce7Sl",0,10-972-846-4255,1358.17,HOUSEHOLD,usly special requests are. pending pin
117797,Customer#000117797,"cOcw50pi1bkyx4,,HS2Y2b4NXkLFA,mjQZXeLFv",0,10-431-973-6984,6303.87,BUILDING,detect daringly ironic instructi
108394,Customer#000108394,bw rRNhVe2tvfFpJB9a CY,0,10-467-647-1870,7137.06,HOUSEHOLD,ckages nag carefully fluffily final dep
127725,Customer#000127725,"LIQb,ij,6ZOkuptOi Fcty0xZEFnkeMbWi7eUaPV",0,10-881-512-6734,909.83,MACHINERY,s above the furiously final requests cajole carefully final asymptotes. furiously
127844,Customer#000127844,AWJKSDwvdo,0,10-662-280-6256,6500.84,BUILDING,final platelets nag fluffily
119330,Customer#000119330,JphhjZvQLsc7rMs3oW6Si,0,10-654-253-6591,9013.26,FURNITURE,its. sometimes regular accounts along the even packages cajole fluffily ironic platelets. ideas wake furiou
114742,Customer#000114742,5slU 9FYHcgT3PO,0,10-753-239-2661,3345.41,HOUSEHOLD,"ven requests use busily unusual, regular multipliers. even Tiresias about the ironic gifts wa"
120354,Customer#000120354,OtnwwuWgNjADKWPzN6qQjH1,0,10-735-998-7580,8007.67,FURNITURE,e finally accounts: regular deposits wake. even waters are fluffily courts. carefully ironic pinto beans b
104101,Customer#000104101,ltKfOErZSFmY9HhqYaGyoS0bSxIdgIF,0,10-798-988-9380,5057.95,AUTOMOBILE,"ual, express theodolites believe enticingly final ideas. ruthlessly express"
114882,Customer#000114882,g4 ClPJmL83Arxiypsh,0,10-179-964-9090,7954.57,BUILDING,g to the blithely ironic accounts. bold theodolites wake carefully final asymptotes. perma
